In [1]:
# do this if you use a custom virtual environment to specify the directory to look for installed packages
import sys
import os
## goes back to the project directory
os.chdir("..")
# switch to the name of your virtual environment
kernel_name = ".venv_mp"
sys.path.append("\\".join([os.getcwd(), kernel_name, "Lib\\site-packages"]))

In [2]:
from data_reader import read_data
from node import get_node_data, standardise_node_data
import pandas as pd
from itertools import combinations as comb
from modelling import grid_search_models, fit_clf, score_clf, select_model_data

In [3]:
raw_data = read_data()
nodes = ["pi2", "pi3", "pi4", "pi5"]

In [6]:
for c in comb(nodes, 2):
    print(c)

('pi2', 'pi3')
('pi2', 'pi4')
('pi2', 'pi5')
('pi3', 'pi4')
('pi3', 'pi5')
('pi4', 'pi5')


In [4]:
def get_data(raw_data, standardised):
    exp_node_data = {}
    for experiment in range(1,4):
        node_data = get_node_data(raw_data, experiment, filtered=False)
        if standardised:
            node_data = standardise_node_data(experiment,node_data)
        a, b, c, d = node_data
        exp_node_data[experiment] = select_model_data(node_data, nodes)
    return exp_node_data

In [5]:
original = get_data(raw_data, standardised=False)
std = get_data(raw_data, standardised=True)
data = [original, std]

In [11]:
std = ["original", "standardised"]
for i in range(2):
    print(std[i])
    for experiment in range(1,4):
        print(f"Experiment {experiment}", end=": ")
        d = data[i][experiment]
        for kernel in ["svr", "lsvr"]:
            models,  df = grid_search_models(kernel, d, nodes)
            for x,y in comb(nodes, 2):
                ex = fit_clf(models[x], d[x])
                exy = score_clf(models[x], d[y])

                ey = fit_clf(models[y], d[y])
                eyx = score_clf(models[y], d[x])
                
                score_x = round(exy/ey, 2)
                score_y = round(eyx/ex, 2)
                if max(score_y, score_x) > 0.8 and max(score_y, score_x) < 1:
                    print(f"({x}, {y} {score_x, score_y})", end = " ")
            print(f"({kernel})", end = ", ")
        print()

original
Experiment 1: (pi2, pi3 (0.85, -2.02)) (pi2, pi4 (0.92, 0.96)) (svr), (pi2, pi3 (0.83, 0.74)) (pi2, pi5 (0.77, 0.87)) (pi3, pi4 (0.69, 0.93)) (pi4, pi5 (0.26, 0.9)) (lsvr), 
Experiment 2: (svr), (lsvr), 
Experiment 3: (svr), (pi2, pi4 (0.94, 0.97)) (lsvr), 
standardised
Experiment 1: (pi2, pi3 (0.93, 0.95)) (pi2, pi4 (0.94, 0.95)) (pi3, pi4 (0.96, 0.97)) (svr), (pi2, pi5 (0.65, 0.9)) (pi3, pi5 (0.75, 0.91)) (pi4, pi5 (0.77, 0.92)) (lsvr), 
Experiment 2: (svr), (lsvr), 
Experiment 3: (svr), (pi3, pi5 (0.81, 0.67)) (pi4, pi5 (0.99, 0.77)) (lsvr), 
